# Développement une API : 

API : 
En informatique, une interface de programmation d’application ou interface de programmation applicative souvent désignée par le terme API pour « Application Programming Interface » en anglais, est un ensemble normalisé de classes, de méthodes, de fonctions et de constantes qui sert de façade par laquelle un logiciel offre des services à d'autres logiciels. 

API REST : 
Une API REST est une interface de programmation des applications (API) conforme aux principes de conception du style d’architecture REST (Representational State Transfer), un style utilisé pour connecter des systèmes hypermédias distribués.
demande un URI
Ce style (REST) est né en 2000 dans la thèse de l’informaticien Roy Fielding.
Les développeurs peuvent mettre au point des API REST en utilisant pratiquement n’importe quel langage de programmation, ces API pouvant prendre en charge une grande variété de formats de données. 
La seule exigence est de respecter les six principes de conception REST suivants, également appelés contraintes architecturales.

Pour qu’une API soit dite « RESTful », elle doit respecter plusieurs contraintes architecturales : 

- Interface uniforme — Toutes les demandes vers une même ressource doivent avoir la même structure. Chaque ressource (par exemple un utilisateur, un produit, un article…) est identifiée de façon unique via un **URI (Uniform Resource Identifier)** (autres **URL (Uniform Resource Locator)**, **URN (Uniform Resource Name)**)

- Découplage client-serveur — Le client (application qui fait la requête) et le serveur (application qui détient la ressource) sont indépendants. Le client ne doit connaître que l’URI de la ressource, et la communication se fait via des requêtes HTTP. 

- Stateless / sans état — Chaque requête envoyée à l’API doit contenir toutes les informations nécessaires : le serveur ne garde pas de “session” liée au client entre plusieurs requêtes. 

- Capacité de mise en cache — Les réponses peuvent être mises en cache (client ou serveur) quand c’est pertinent, ce qui améliore les performances et la scalabilité. 

- Architecture en couches — Les communications peuvent passer par plusieurs couches (serveurs intermédiaires, passerelles, etc.), sans que le client ou le serveur final sachent s’ils communiquent directement ou via un intermédiaire. 
  
- Optionnel : Code à la demande — Dans certains cas, l’API pourrait envoyer du code exécutable au client (par exemple un script), mais ce principe est rarement utilisé.

FAST API : 
Bibliothèque python pour faire une API. 
une API tourne sur un serveur 

DJANGO : 

Décorateur : 
@aaaa
exemple : 
```
python
def timer_decorator(function):
    def wrapper(*args,**kwargs):
        start = time.perf.counter()
        result = function(*arg,**kwargs)
        end = time.perf.counter()
        print(f'La fonction {function.__name__} a été executée en {end - start} secondes')
        return result
    return wrapper
```


In [ ]:
import pandas as pd
import os
from loguru import logger

CSV_FILE_PATH = os.path.join("DEV", "quotes_db.csv")
def write_db (df : pd.DataFrame):
    df.to_csv(
        path_or_buf=CSV_FILE_PATH,
        index=True,
        index_label= 'id',
        )

def read_db (df : pd.DataFrame):
    pd.read_csv(
        filepath_or_buffer=CSV_FILE_PATH,
        index_col= 'id',
        )
    return df

def initialise_db():
    if os.path.exists(CSV_FILE_PATH):
        logger.info("LA base de donnée existe")
    else : 
        logger.info(f"Impossible de trouver le fichier {CSV_FILE_PATH}")
        df = pd.DataFrame(columns=['id', 'text'])
        df = df.set_index('id')
        write_db(df)
        logger.info(f"le fichier{CSV_FILE_PATH} a été créé")

initialise_db()

PermissionError: [Errno 13] Permission denied: '/quotes_db.csv'

In [24]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column, Integer, String
from loguru import logger

Base = declarative_base()

class CITATIONS(Base):
    __tablename__ = 'citations'
    # clé primaire
    id = Column(Integer, primary_key=True, autoincrement=True)  
    text = Column(String)
    

DB_FILE_PATH = os.path.join("backend", "data", "quotes_db.db")
ENGINE = create_engine(f"sqlite:///{DB_FILE_PATH}", echo=True)

def create_session(engine = ENGINE):
    '''
    Create and return a new SQLAlchemy session.

    Args : 
        engine : sqlalchemy.Engine
            The SQLAlchemy engine connected to the database.

    Returns : 
        Session
            A new SQLAlchemy session instance.
    '''
    try : 
        # create the tables
        Base.metadata.create_all(engine)
        # open the session
        Session = sessionmaker(bind=engine)
        return Session()
    except Exception as e:
        print(f"Error creating database session: {e}")

df = pd.read_csv(
    filepath_or_buffer="backend/data/quotes_db.csv",
    index_col= 'id',
    )
def write_db (df : pd.DataFrame):
    """
    Write a csv file from a pandas DataFrame
    
    :param df: DataFrame with data you want to write in a file
    :type df: pd.DataFrame

    :returns: None
    """
    session = create_session()
    citations_to_add = []
    for idx, row in df.iterrows():
        citation_id = idx
        text = str(row['text'])
        citation = CITATIONS(id=citation_id, text=text)
        citations_to_add.append(citation)
    
        session.add_all(citations_to_add)
    session.commit()
    session.close()


def read_db():
    """
    Read a csv file from the path CSV_FILE_PATH

    :returns: DataFrame with data you read from the file
    :type df: pd.DataFrame
    """
    session = create_session()
    db = [
        {"id" : p.id , "text" : p.text}
        for p in session.query(CITATIONS).all()
    ]
    df = pd.DataFrame(db)
    # df = check_db(df)
    return df

read_db()

2025-12-09 08:47:13,542 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-09 08:47:13,543 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("citations")
2025-12-09 08:47:13,543 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-09 08:47:13,544 INFO sqlalchemy.engine.Engine COMMIT
2025-12-09 08:47:13,546 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-09 08:47:13,547 INFO sqlalchemy.engine.Engine SELECT citations.id AS citations_id, citations.text AS citations_text 
FROM citations
2025-12-09 08:47:13,548 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()


,id,text
0,1,"Caractéristique qui, pour tout ensemble, fini ..."
1,2,p
2,3,ab
3,4,ab
4,5,ajout
5,6,ab
6,7,test
7,8,ab
8,9,nan
9,10,titi
